Строим запрос к нашему сервису

In [ ]:
import requests

import urllib.request
import json 

In [ ]:
# Пример данных
data = ( 
    "Stylect is a dynamic startup that helps helps women discover and buy shoes. We’re a small team based in London that has previously worked at Google, Techstars, Pixelmator and Rocket Internet.We place a high premium on simplicity no matter what we’re working on (i.e. design, programming, marketing). We’re also a team that ships fast. We built version 1 of our app in a week, the next release (built in a month) was featured in the Apple Appstore Italy as a best new fashion app. Fast release cycles are challenging, but also very fun - which is why we love them.\xa0As we’ve grown, the projects that we’re working on have grown both in scale and in technical complexity. \xa0Stylect is looking for someone who can help us improve our backend which gathers product data; analyses/categorizes it; and shows it to thousands of users daily. Each step in the process has unique challenges that demands a strong technical background.",
    "ustwo offers you the opportunity to be yourself, whilst delivering the best work on the planet for some of the biggest and most innovative brands. A culture thriving on collaboration underpins what is an amazing work smart/ live well environment.We genuinely care about the work that we deliver and the people who help make it all possible. We only invest in projects, people and practices that we believe in, to ensure we remain excited about every opportunity.",
    "We are negotiable on salary and there is the potential for equity for the right candidate."
)

In [ ]:
# формируем запрос
def send_json(x):
    description, company_profile, benefits = x
    #print(description, company_profile, benefits)
    body = {
        'description': description, 
        'company_profile': company_profile,
        'benefits': benefits
        }
    myurl = 'http://3669-104-198-71-64.ngrok.io' + '/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

In [ ]:
# обращение к серверу с запросом из одного набора (его построили руками выше - data)
response = send_json(data)
print('предсказание', response)

предсказание 0.001129359819418078


Сделаем обработку массы запросов:

1. загрузим данные X_test

https://drive.google.com/file/d/1-Ltk-t1acAoA5Xtz0zPZ1yOqjoYiP2h5

https://drive.google.com/file/d/1vb0KLnOeRsg4G33TaPxKEV2OMnla1oyL

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1-Ltk-t1acAoA5Xtz0zPZ1yOqjoYiP2h5' -O X_test.csv
!wget 'https://drive.google.com/uc?export=download&id=1vb0KLnOeRsg4G33TaPxKEV2OMnla1oyL' -O y_test.csv

--2022-04-16 16:37:22--  https://drive.google.com/uc?export=download&id=1-Ltk-t1acAoA5Xtz0zPZ1yOqjoYiP2h5
Resolving drive.google.com (drive.google.com)... 64.233.189.139, 64.233.189.113, 64.233.189.102, ...
Connecting to drive.google.com (drive.google.com)|64.233.189.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/horo989ttv7s8aqbr3b7gvio6r557o90/1650126975000/14904333240138417226/*/1-Ltk-t1acAoA5Xtz0zPZ1yOqjoYiP2h5?e=download [following]
--2022-04-16 16:37:25--  https://doc-0k-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/horo989ttv7s8aqbr3b7gvio6r557o90/1650126975000/14904333240138417226/*/1-Ltk-t1acAoA5Xtz0zPZ1yOqjoYiP2h5?e=download
Resolving doc-0k-c0-docs.googleusercontent.com (doc-0k-c0-docs.googleusercontent.com)... 74.125.203.132, 2404:6800:4008:c03::84
Connecting to doc-0k-c0-docs.googleusercontent.com (doc-0k-c0-docs

In [ ]:
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score
from urllib import request, parse


X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

2. Передаем запрос одиночный из наших данных и посмотрим на ответ

In [ ]:
response = send_json(X_test[['description', 'company_profile', 'benefits']].iloc[0,:])

In [ ]:
response

0.02602202506963531

In [ ]:
X_test[['description', 'company_profile', 'benefits']].head()

,description,company_profile,benefits
0,Stylect is a dynamic startup that helps helps ...,NaN,We are negotiable on salary and there is the p...
1,General Summary: Achieves maximum sales profit...,NaN,Great Health and DentalFast Advancement Opport...
2,"At ustwo™ you get to be yourself, whilst deliv...",ustwo offers you the opportunity to be yoursel...,NaN
3,About EDITDEDITD runs the world's biggest appa...,"We build software for fashion retailers, to he...",NaN
4,As a Web Engineer at Runscope you'll be respon...,Runscope is building tools for developers work...,Be a part of an experienced team who have work...


3. Сделаем N запросов и оценим время

In [ ]:
N = 50

In [ ]:
%%time
predictions = X_test[
                     ['description', 'company_profile', 'benefits']
                     ].iloc[:N].apply(lambda x: send_json(x), axis=1)

CPU times: user 267 ms, sys: 29.4 ms, total: 297 ms
Wall time: 20.9 s


In [ ]:
predictions.values[:5]

array([0.02602203, 0.04317015, 0.00370601, 0.00112958, 0.00151454])

4. Посчитаем метрику

In [ ]:
y_test.head()

,fraudulent
0,0
1,0
2,0
3,0
4,0


In [ ]:
import numpy as np

precision, recall, thresholds = precision_recall_curve(y_test[:N], predictions)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.907162655181958, F-Score=0.667, Precision=1.000, Recall=0.500


In [ ]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

0.96875

5. Анализ : в целом похоже